In [7]:
class Player:
    def __init__(self, name, kills, deaths, wins, losses):
        self.name = name
        self.kills = kills
        self.deaths = deaths
        self.wins = wins
        self.losses = losses

p1 = Player('Cat', 2, 2, 2, 2)
p1.kills += 11
p1.kills

13

In [ ]:
import pickle

def save_object(obj, fname):
    try:
        with open(fname, "wb") as f:
            pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)
    except Exception as ex:
        print("Error during pickling object (Possibly unsupported):", ex)


def load_object(filename):
    try:
        with open(filename, "rb") as f:
            return pickle.load(f)
    except Exception as ex:
        print("Error during unpickling object (Possibly unsupported):", ex)


try: 
    load_object('monkey.pickle')
except:
    monkey = []
    save_object(monkey, 'monkey.pickle')
